In [223]:
import requests
from bs4 import BeautifulSoup

# Headers for the HTTP request
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:123.0) Gecko/20100101 Firefox/123.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.airlinequality.com/airline-reviews/british-airways/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
}

# URL and page settings
url = 'https://www.airlinequality.com/airline-reviews/british-airways/page/{}/?sortby=post_date%3ADesc&pagesize=4000'
page = 1

# Fetch the webpage content
response = requests.get(url.format(page), headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
reviews = soup.find_all("article", attrs={'itemprop': 'review'})

In [236]:
# Helper function to extract star ratings
def extract_ratings(review, class_name):
    ratings = []
    table = review.table
    if table:
        td_elements = table.find_all("td", class_=class_name)
        if td_elements:
            for td in td_elements:
                next_sibling = td.find_next_sibling()
                if next_sibling:
                    filled_stars = next_sibling.find_all('span', class_='star fill')
                    if filled_stars:
                        last_filled_star = filled_stars[-1].get_text(strip=True)
                        ratings.append(int(last_filled_star))
                    else:
                        ratings.append(0)
                else:
                    ratings.append(0)
        else:
            ratings.append(0)
    else:
        ratings.append(0)
    return ratings

In [237]:
# Define the review categories
categories = [
    "seat_comfort",
    "ground_service",
    "value_for_money",
    "cabin_staff_service",
    "food_and_beverages",
    "inflight_entertainment",
    "wifi_and_connectivity",
    "recommended"
]

ratings_dict = {category: [] for category in categories}

In [238]:
# Extract ratings for each category
for review in reviews:
    for category in categories:
        ratings = extract_ratings(review, category)
        ratings_dict[category].extend(ratings)

In [239]:
# Process 'recommended' category separately as it does not follow the star rating pattern
recommended = []
for review in reviews:
    table = review.table
    if table:
        td_elements = table.find_all("td", class_="recommended")
        for td in td_elements:
            next_sibling = td.find_next_sibling()
            if next_sibling:
                review_value = next_sibling.get_text(strip=True).lower()
                if review_value == "no":
                    recommended.append(0)
                else:
                    recommended.append(1)
    else:
        recommended.append(0)
ratings_dict["recommended"] = recommended

In [241]:
# Print the collected ratings
for category, ratings in ratings_dict.items():
    print(f'{category}: {len(ratings)}')

seat_comfort: 3807
ground_service: 3807
value_for_money: 3807
cabin_staff_service: 3807
food_and_beverages: 3807
inflight_entertainment: 3807
wifi_and_connectivity: 3807
recommended: 3807
